In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
import requests
from bs4 import BeautifulSoup
from itertools import combinations
from datetime import datetime, timedelta
import pandas as pd

## Topsport

In [11]:
url = "https://en.topsport.lt/basketball/nba"
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')
today = datetime.today()

events = soup.find_all('div', {'class': 'js-prelive-event-row'})

matches_topsport = []
for event in events:
    try:
        date = event.find('span', {'class':'prelive-event-date'}).text.lower()
        teams = event.find_all('div', {'class': 'prelive-outcome-buttons-item-title-overflow'})
        odds = event.find_all('span', "prelive-list-league-rate ml-1 h-font-secondary h-fs17 h-fw500")
        # Convert "Today" and "Tomorrow" to actual dates
        if "today" in date:
            date = today.strftime("%Y-%m-%d ") + date.split(" ")[1]
        elif "tomorrow" in date:
            tomorrow = today + timedelta(days=1)
            date = tomorrow.strftime("%Y-%m-%d ") + date.split(" ")[1]
        # Ensure we have both teams and their respective odds
        if len(teams) == 2 and len(odds) == 2:
            team1 = teams[0].text.strip()
            team2 = teams[1].text.strip()
            odd1 = float(odds[0].text)
            odd2 = float(odds[1].text)
            
            if team1 == "Yes" or team2 == "Yes": # Skip extra bets with "yes" "no" options
                continue

            matches_topsport.append((date, (team1, team2), (odd1, odd2)))
    except Exception as e:
        print(f"An error occurred: {e}")
        continue

## Betsafe

In [12]:
# Set options
options = Options()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")  # Disable GPU acceleration
options.add_argument("--no-sandbox")  # Disables sandbox for the browser
options.add_argument("--disable-images")  # Disable images

# Start WebDriver
driver = webdriver.Firefox(options=options)
driver.get("https://www.betsafe.lt/en/betting/nba")

try:
    # Wait for the events to load
    events = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "wpt-table__row"))
    )
    
    # List to store all matches
    matches_betsafe = []
    # Loop through events
    for event in events:
        # Extract match date and time
        date_elements = event.find_elements(By.CLASS_NAME, "wpt-time")
        if date_elements:
            date_text = date_elements[0].text  # e.g., 04 Dec\n20:00
            date_text = date_text.replace('\n', ' ')

            # Convert to datetime object
            date = datetime.strptime(date_text, "%d %b %H:%M")
            date = date.strftime("%m-%d %H:%M") # Output: e.g., 12-04 20:00
        
        
        teams = event.find_elements(By.CLASS_NAME, "wpt-teams__team")
        if len(teams) != 2:  # Ensure there are exactly 2 teams
            continue

        # Extract odds values
        odds_elements = event.find_elements(By.CLASS_NAME, "wpt-odd-changer")
        odds = [odd.text for odd in odds_elements if odd.text]  # Extract non-empty text
        if not odds:
            continue

        team1 = teams[0].find_element(By.TAG_NAME, "a").text
        team2 = teams[1].find_element(By.TAG_NAME, "a").text

        odd1 = float(odds[0])
        odd2 = float(odds[1])
        matches_betsafe.append((date, (team1, team2), (odd1, odd2)))
        
finally:
    # Close the browser
    driver.quit()

## 7bet

In [13]:
api_url = "https://sb2frontend-altenar2.biahosted.com/api/widget/GetEvents"

params = {
    "culture": "lt-LT",
    "timezoneOffset": "-120",
    "integration": "7bet",
    "deviceType": "1",
    "numFormat": "en-GB",
    "countryCode": "LT",
    "eventCount": "0",
    "sportId": "0",
    "champIds": "2980"
}

# Add headers copied from browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Referer": "https://7bet.lt/", 
    "Origin": "https://7bet.lt",
    "Accept": "application/json, text/plain, */*"
}

response = requests.get(api_url, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")

# Only first 2 bets of each match have False, others True. Can also be used 'typeID'==1 or 3
filtered_events = [(idx, event) for idx, event in enumerate(data['odds']) if event['isMB'] is False]

# Group the filtered events into pairs
grouped_pairs = [filtered_events[i:i+2] for i in range(0, len(filtered_events), 2)]

matches_bet7 = []
# Extract only price and name from each event
for group in grouped_pairs:
    if len(group) == 2:
        # Extracting the first and second team's names and prices
        team1, team2 = group[0][1]['name'], group[1][1]['name']
        odd1, odd2 = group[0][1]['price'], group[1][1]['price']
        
        # Round odds
        odd1 = round(odd1, 2)
        odd2 = round(odd2, 2)
        
        # Append the tuple to the matches list
        matches_bet7.append(((team1, team2), (odd1, odd2)))

# matches_bet7

## Combine all dataframes and calculate arbitrages

In [14]:
df_topsport = pd.DataFrame(
    matches_topsport,
    columns=["date","match","odds"],
)

df_betsafe = pd.DataFrame(
    matches_betsafe,
    columns=["date", "match","odds"],
)

df_7bet = pd.DataFrame(
    matches_bet7,
    columns=["match","odds"],
)

# Define helper function to find matching teams
def merge_odds(df_topsport, df_betsafe, df_7bet):
    # Create an empty list to store the results
    merged_matches = []

    # Keep track of already matched rows
    matched_betsafe_indices = set()
    matched_7bet_indices = set()

    # Iterate over rows in df_topsport
    for index, row in df_topsport.iterrows():
        top_teams = row['match']
        matched_betsafe_odds = None
        matched_7bet_odds = None
        
        # Try to match with betsafe odds
        for bet_index, bet_row in df_betsafe.iterrows():
            if bet_index in matched_betsafe_indices:
                continue  # Skip already matched rows
            bet_teams = bet_row['match']
            if top_teams == bet_teams:
                matched_betsafe_odds= bet_row['odds']
                matched_betsafe_indices.add(bet_index)
                break

        # Try to match with 7bet odds
        for bet_index, bet_row in df_7bet.iterrows():
            if bet_index in matched_7bet_indices:
                continue  # Skip already matched rows
            bet_teams = bet_row['match']
            if top_teams == bet_teams:
                matched_7bet_odds = bet_row['odds']
                matched_7bet_indices.add(bet_index)
                break

        # If a match was found, merge the data
        merged_matches.append(
            {
                'date': row['date'],
                'match': row['match'],
                'topsport_odds': row['odds'],
                'betsafe_odds': matched_betsafe_odds,
                '7bet_odd_1': matched_7bet_odds,
            }
        )

    # Convert the list of matched rows into a DataFrame
    df = pd.DataFrame(merged_matches)
    return df

df = merge_odds(df_topsport, df_betsafe, df_7bet)

In [ ]:
def calculate_arbitrage(odd1, odd2):
    total = 1 / odd1 + 1 / odd2
    bet1 = round((1 / odd1) * 100 / total, 2)
    bet2 = round((1 / odd2) * 100 / total, 2)
    return bet1, bet2

def display_result(match, total_sum, bet1, bet2, odd1, odd2, b1, b2):
    return (
        f'Arbitrage found in match {match} with sum={total_sum}\n'
        f'Place bet {bet1}% on Team 1 in {b1} at {odd1}, \n'
        f'Place bet {bet2}% on Team 2 in {b2} at {odd2}'
    )

def check_arbitrage(row):
    # Extract odds
    odds_top = row['topsport_odds']
    odds_betsafe = row['betsafe_odds']
    odds_7bet = row['7bet_odd_1']
    
    # Create a list of tuples (bookmaker, odds pair)
    odds_list = []
    if odds_top is not None:
        odds_list.append(('topsport', odds_top))
    if odds_betsafe is not None:
        odds_list.append(('betsafe', odds_betsafe))
    if odds_7bet is not None:
        odds_list.append(('7bet', odds_7bet))
    results = []

    # Generate all combinations of odds from different bookmakers (pairing first and second odds)
    for pair in combinations(odds_list, 2):
        bookmaker1, (odd11, odd12) = pair[0]
        bookmaker2, (odd21, odd22) = pair[1]
        
        # Generate all combinations of first odds of one bookmaker with second odds of the other
        conditions = [
            (round(1 / odd11 + 1 / odd22, 5), bookmaker1, bookmaker2, odd11, odd22),
            (round(1 / odd12 + 1 / odd21, 5), bookmaker1, bookmaker2, odd12, odd21),
            (round(1 / odd11 + 1 / odd12, 5), bookmaker1, bookmaker1, odd11, odd12),
            (round(1 / odd21 + 1 / odd22, 5), bookmaker2, bookmaker2, odd21, odd22),
        ]
        # Check each condition for arbitrage
        min_sum = 2
        for sum, b1, b2, odd1, odd2 in conditions:
            if sum < 1.03:  # Arbitrage condition. Should be 1!!
                min_sum = min(sum, min_sum)
                bet1, bet2 = calculate_arbitrage(odd1, odd2)
                result = display_result(row['match'], sum, bet1, bet2, odd1, odd2, b1, b2)
                results.append(result)
    # Return the count of unique arbitrage opportunities
    return (results, min_sum) if results else (False, None)

# Apply the arbitrage check to the DataFrame
df[['arbitrage', 'min_sum']] = df.apply(check_arbitrage, axis=1, result_type='expand')
df

,date,match,topsport_odds,betsafe_odds,7bet_odd_1,arbitrage,min_sum
0,2024-12-11 02:00,"(Milwaukee Bucks, Orlando Magic)","(1.34, 3.3)","(1.32, 3.4)","(1.36, 3.25)","[Arbitrage found in match ('Milwaukee Bucks', ...",1.02941
1,2024-12-11 04:30,"(Oklahoma City Thunder, Dallas Mavericks)","(1.54, 2.48)","(1.5, 2.6)","(1.56, 2.5)",[Arbitrage found in match ('Oklahoma City Thun...,1.02564
2,2024-12-12 02:00,"(New York Knicks, Atlanta Hawks)","(1.36, 3.2)","(1.32, 3.4)","(1.36, 3.25)","[Arbitrage found in match ('New York Knicks', ...",1.02941
3,2024-12-12 04:30,"(Houston Rockets, Golden State Warriors)","(1.73, 2.12)","(1.67, 2.2)","(1.74, 2.15)","[Arbitrage found in match ('Houston Rockets', ...",1.02926
4,2024-12-13 02:30,"(Boston Celtics, Detroit Pistons)","(1.1, 7.0)",None,"(1.11, 6.75)",False,NaN
5,2024-12-13 02:30,"(Miami Heat, Toronto Raptors)","(1.17, 5.2)",None,"(1.17, 5.25)",False,NaN
6,2024-12-13 03:00,"(New Orleans Pelicans, Sacramento Kings)","(2.85, 1.44)",None,"(2.8, 1.44)",False,NaN
7,2024-12-25 19:00,"(New York Knicks, San Antonio Spurs)","(1.26, 3.95)",None,"(1.25, 4.0)",False,NaN
8,2024-12-25 21:30,"(Dallas Mavericks, Minnesota Timberwolves)","(1.59, 2.37)",None,"(1.61, 2.35)",False,NaN
9,2024-12-26 00:00,"(Boston Celtics, Philadelphia 76ers)","(1.25, 4.0)",None,"(1.27, 3.85)",False,NaN
